In [4]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools

from rdkit.Chem import AllChem as Chem
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from rdkit import Chem
from collections import namedtuple

def merge(mol, marked, aset):
    bset = set()
    for idx in aset:
        atom = mol.GetAtomWithIdx(idx)
        for nbr in atom.GetNeighbors():
            jdx = nbr.GetIdx()
            if jdx in marked:
                marked.remove(jdx)
                bset.add(jdx)
    if not bset:
        return
    merge(mol, marked, bset)
    aset.update(bset)

# atoms connected by non-aromatic double or triple bond to any heteroatom
# c=O should not match (see fig1, box 15).  I think using A instead of * should sort that out?
PATT_DOUBLE_TRIPLE = Chem.MolFromSmarts('A=,#[!#6]')
# atoms in non aromatic carbon-carbon double or triple bonds
PATT_CC_DOUBLE_TRIPLE = Chem.MolFromSmarts('C=,#C')
# acetal carbons, i.e. sp3 carbons connected to tow or more oxygens, nitrogens or sulfurs; these O, N or S atoms must have only single bonds
PATT_ACETAL = Chem.MolFromSmarts('[CX4](-[O,N,S])-[O,N,S]')
# all atoms in oxirane, aziridine and thiirane rings
PATT_OXIRANE_ETC = Chem.MolFromSmarts('[O,N,S]1CC1')

PATT_TUPLE = (PATT_DOUBLE_TRIPLE, PATT_CC_DOUBLE_TRIPLE, PATT_ACETAL, PATT_OXIRANE_ETC)

def identify_functional_groups(mol):
    marked = set()
#mark all heteroatoms in a molecule, including halogens
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() not in (6,1): # would we ever have hydrogen?
            marked.add(atom.GetIdx())

#mark the four specific types of carbon atom
    for patt in PATT_TUPLE:
        for path in mol.GetSubstructMatches(patt):
            for atomindex in path:
                marked.add(atomindex)

#merge all connected marked atoms to a single FG
    groups = []
    while marked:
        grp = set([marked.pop()])
        merge(mol, marked, grp)
        groups.append(grp)

#extract also connected unmarked carbon atoms
    ifg = namedtuple('IFG', ['atomIds', 'atoms', 'type'])
    ifgs = []
    for g in groups:
        uca = set()
        for atomidx in g:
            for n in mol.GetAtomWithIdx(atomidx).GetNeighbors():
                if n.GetAtomicNum() == 6:
                    uca.add(n.GetIdx())
        ifgs.append(ifg(atomIds=tuple(list(g)), atoms=Chem.MolFragmentToSmiles(mol, g, canonical=True), type=Chem.MolFragmentToSmiles(mol, g.union(uca),canonical=True)))
    return ifgs

def main():
    for ix, smiles in enumerate([
        'CSc1nc(NC(C)C)nc(NC(C)C)n1',
 'CCC(C)(C)CC',
 'CCCC(C)C',
 'CNc1ccccc1',
 'C=CCCCCCCCC',
 'Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)Cl)cc1',
 'COP(=S)(OC)SCC(=O)N(C)C=O',
 'Clc1ccccc1-c1cc(Cl)c(Cl)c(Cl)c1Cl',
 'FC(F)(Cl)C(F)(Cl)Cl',
 'Clc1cc(Cl)c(Cl)c(-c2cc(Cl)cc(Cl)c2Cl)c1',
 'Clc1cc(Cl)c(Cl)c(-c2cc(Cl)cc(Cl)c2Cl)c1',
 'Nc1ccccc1Cl',
 'Clc1ccc(Cl)c(-c2cccc(Cl)c2Cl)c1',
 'Nc1ccc(C(=O)O)cc1',
 'CC1=C(C(=O)Nc2ccccc2)S(=O)(=O)CCO1',
 'CCCCCO',
 'COc1ccc(Oc2ccc(NC(=O)N(C)C)cc2)cc1',
 'CCOP(=S)(CC)Oc1cc(Cl)c(Cl)cc1Cl',
 'CCCCCCOC(C)=O',
 'Cc1ccc(C)c2ccccc12',
 'CCC(O)CC',
 'Cc1c2ccccc2c(C)c2c1ccc1ccccc12',
 'Clc1ccc(Oc2ccc(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'CCC',
 'CCC',
 'Clc1cccc(-c2ccc(Cl)c(Cl)c2Cl)c1Cl',
 'Cc1cc(C)cc(C)c1',
 'CC(C)COC=O',
 'CCCCOC(=O)c1ccccc1C(=O)OCC(CC)CCCC',
 'c1ccc2cnccc2c1',
 'c1ccc(C2CO2)cc1',
 'CC(C)OC(=O)C(O)(c1ccc(Cl)cc1)c1ccc(Cl)cc1',
 'O=C1C=CC(=O)C=C1',
 'O=C1C=CC(=O)C=C1',
 'COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1',
 'C=CC(=O)OC',
 'c1cc2ccc3cccc4ccc(c1)c2c34',
 'c1cc2ccc3cccc4ccc(c1)c2c34',
 'CCCCCCCO',
 'CCCCc1c(C)nc(NCC)[nH]c1=O',
 'CCCCc1c(C)nc(NCC)nc1O',
 'FC1(F)C(F)(F)C(F)(F)C1(F)F',
 'CCNc1nc(NC(C)(C)C)nc(OC)n1',
 'FC(F)(F)C(Cl)Br',
 'CCCO',
 'O=C(O)CCCCCCCC(=O)O',
 'O=[N+]([O-])c1ccccc1Cl',
 'CCCCCCCCCCCCCC',
 'C=CCBr',
 'ClC1=C(Cl)C2(Cl)C3C(Cl)C(Cl)CC3C1(Cl)C2(Cl)Cl',
 'ClC1=C(Cl)C2(Cl)C3C(Cl)C(Cl)CC3C1(Cl)C2(Cl)Cl',
 'CCCCC(CC)COC(=O)c1ccccc1C(=O)OCC(CC)CCCC',
 'CCCCC(CC)COC(=O)c1ccccc1C(=O)OCC(CC)CCCC',
 'CCCCCCCCC',
 'CC=C(C)C',
 'CN(C)C(=O)Nc1ccc(Cl)cc1',
 'CCSC(=O)N(CC(C)C)CC(C)C',
 'CCCCCCN',
 'ClC(Cl)C(Cl)(Cl)Cl',
 'Cc1cc(C)c(O)c(C)c1',
 'Clc1cc(Cl)c(Cl)c(-c2cc(Cl)c(Cl)cc2Cl)c1',
 'ClCC1CO1',
 'CCC(C)Cl',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'CCC(C)O',
 'O=[N+]([O-])c1ccc(O)cc1',
 'CCOP(=S)(OCC)SCSCC',
 'COP(=O)(OC)OC(Br)C(Cl)(Cl)Br',
 'CCOP(=S)(OCC)Oc1ccc2c(C)c(Cl)c(=O)oc2c1',
 'CCOP(=S)(OCC)Oc1ccc2c(C)c(Cl)c(=O)oc2c1',
 'CCCCCCCCCCCCCCCCO',
 'CCCCCCCCC(=O)OCC',
 'C#CC(C)N(C)C(=O)Nc1ccc(Cl)cc1',
 'C=CCl',
 'CC(C)CC(C)C',
 'COC(C)(C)C',
 'Clc1ccc(-c2ccc(Cl)cc2Cl)cc1',
 'CCCCCC(C)=O',
 'CCCCCCCCCCCCCCCCCCCC',
 'CCOc1cc(C=O)ccc1O',
 'C=CC(C)(O)CCC=C(C)C',
 'Clc1cc(Cl)c(Cl)c(Cl)c1Cl',
 'Clc1cc(Cl)c(Cl)c(Cl)c1Cl',
 'C=C(C)C(=O)O',
 'CCSSCC',
 'COc1ccccc1[N+](=O)[O-]',
 'Cc1cc(O)ccc1Cl',
 'C=CC(=O)OCC(C)C',
 'ClCC(Cl)CCl',
 'C#CCCCC',
 'Clc1ccc2ccccc2c1',
 'CC(C)Oc1cc(-n2nc(C(C)(C)C)oc2=O)c(Cl)cc1Cl',
 'C1=CC=CCC=C1',
 'Clc1c(Cl)c(Cl)c(Oc2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'CCCCc1c(C)nc(N(C)C)nc1O',
 'Clc1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl',
 'Clc1ccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c1',
 'Clc1ccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c1',
 'Cc1c2ccccc2cc2ccccc12',
 'CCOc1ccc(NC(C)=O)cc1',
 'CCOc1ccc(NC(C)=O)cc1',
 'CCOc1ccc(NC(C)=O)cc1',
 'CCC(C)(C)Cl',
 'N#Cc1c(Cl)c(Cl)c(Cl)c(C#N)c1Cl',
 'N#Cc1c(Cl)c(Cl)c(Cl)c(C#N)c1Cl',
 'c1cc2cccc3c4cccc5cccc(c(c1)c23)c54',
 'c1cc2cccc3c4cccc5cccc(c(c1)c23)c54',
 'CCC(CO[N+](=O)[O-])O[N+](=O)[O-]',
 'CC1=CCCCC1',
 'Cc1ccccc1C',
 'CCC(C)n1c(=O)[nH]c(C)c(Br)c1=O',
 'Clc1cc(Cl)c(-c2cc(Cl)cc(Cl)c2Cl)c(Cl)c1',
 'CNC(=O)Oc1cc(C)cc(C)c1',
 'COP(=S)(Oc1cc(Cl)c(Br)cc1Cl)c1ccccc1',
 'Clc1ccc(C(c2ccccc2Cl)C(Cl)(Cl)Cl)cc1',
 'CNC(=O)Oc1cccc2ccccc12',
 'CNC(=O)Oc1cccc2ccccc12',
 'CNC(=O)Oc1cccc2ccccc12',
 'CC(C)OC(=O)Nc1cccc(Cl)c1',
 'CC(C)OC(=O)Nc1cccc(Cl)c1',
 'c1cc2c3c(cccc3c1)CC2',
 'CCCCOC',
 'Clc1ccc(-c2c(Cl)ccc(Cl)c2Cl)c(Cl)c1',
 'CCCC(C)(C)C',
 'CCC(Cl)CC',
 'c1ccc2c(c1)CCCC2',
 'CCCBr',
 'ClC(Cl)=C(Cl)Cl',
 'O=[N+]([O-])c1ccc(Cl)cc1',
 'c1ccc(Cc2ccccc2)cc1',
 'c1ccc(Cc2ccccc2)cc1',
 'CCCCCCCCCCCCCCCC',
 'Clc1ccc(Cl)c(-c2ccccc2Cl)c1',
 'FC(F)(Cl)C(F)(F)Cl',
 'COCCCNc1nc(NC(C)C)nc(SC)n1',
 'CCSC(=O)N1CCCCCC1',
 'OC(CCl)CCl',
 'ClC1=C(Cl)C2(Cl)C3C4CC(C5OC45)C3C1(Cl)C2(Cl)Cl',
 'ClC1=C(Cl)C2(Cl)C3C4CC(C5OC45)C3C1(Cl)C2(Cl)Cl',
 'CCSC(=O)N(CC)C1CCCCC1',
 'CC(CCO[N+](=O)[O-])O[N+](=O)[O-]',
 'CCOC(C)OCC',
 'Clc1ccc(Oc2ccc(Cl)c(Cl)c2)cc1Cl',
 'C1CCOCC1',
 'BrC(Br)Br',
 'CCC(C)c1ccccc1OC(=O)NC',
 'Clc1ccc(-c2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c1',
 'Clc1c(Cl)c(Cl)c2c(c1Cl)Oc1ccccc1O2',
 'CCCCOC(C)=O',
 'CC(N)=O',
 'CCOc1ccccc1',
 'c1ccc(Nc2ccccc2)cc1',
 'C1CCCCCC1',
 'c1ccc(N=Nc2ccccc2)cc1',
 'c1ccc(N=Nc2ccccc2)cc1',
 'CCS',
 'CCOP(=S)(OCC)SCCSCC',
 'CCCCOC(=O)c1ccccc1C(=O)OCCCC',
 'CCCCOC(=O)c1ccccc1C(=O)OCCCC',
 'Cc1cccc2c1ccc1ccccc12',
 'Nc1ccccc1[N+](=O)[O-]',
 'Cc1cccc2sc3nncn3c12',
 'CCOP(=O)(OCC)OCC',
 'CCCCc1c(C)nc(NCC)nc1OS(=O)(=O)N(C)C',
 'CCc1ccc(CC)cc1',
 'COc1ccc(NC(=O)N(C)C)cc1Cl',
 'CC12CCC(CC1=O)C2(C)C',
 'CC12CCC(CC1=O)C2(C)C',
 'Clc1cc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c(Cl)c1Cl',
 'CNC(=O)Oc1cccc2c1OC(C)(C)C2',
 'CNC(=O)Oc1cccc2c1OC(C)(C)C2',
 'CCCCOCCCC',
 'Clc1cc(Cl)c(Oc2cc(Cl)c(Cl)c(Cl)c2)cc1Cl',
 'c1ccc2nc3ccccc3cc2c1',
 'CCCCCCCCl',
 'OCC(O)C(O)C(O)C(O)CO',
 'CCCCCCCCC(=O)O',
 'CCC(=O)CC',
 'CCCC(=O)O',
 'CC(=O)CCC(C)C',
 'CC(C)[N+](=O)[O-]',
 'CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1',
 'CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1',
 'BrCBr',
 'O=C(O)c1ccc(O)cc1',
 'O=C(O)c1ccc(O)cc1',
 'CCOC(=O)CC',
 'CCOC(=O)c1ccccc1C(=O)OCC',
 'CCOC(=O)c1ccccc1C(=O)OCC',
 'CC(C)I',
 'c1ccc2c(c1)cc1ccc3cccc4ccc2c1c34',
 'c1ccc2c(c1)cc1ccc3cccc4ccc2c1c34',
 'c1ccc2c(c1)cc1ccc3cccc4ccc2c1c34',
 'ClC(Cl)Br',
 'CC(C)OC(=O)C(O)(c1ccc(Br)cc1)c1ccc(Br)cc1',
 'O=C(O)CCl',
 'Clc1c(Cl)c(Cl)c2c(c1Cl)Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1O2',
 'CSc1nnc(C(C)(C)C)c(=O)n1N',
 'CCc1cccc(CC)c1N',
 'Clc1cc(Oc2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c(Cl)c1Cl',
 'Clc1ccc(-c2ccccc2)cc1',
 'ClC1=C(Cl)C2(Cl)C3C(Cl)C=CC3C1(Cl)C2(Cl)Cl',
 'ClC1=C(Cl)C2(Cl)C3C(Cl)C=CC3C1(Cl)C2(Cl)Cl',
 'COc1ccc(Cl)c(Cl)c1Cl',
 'Cc1ccc(C)c2c(C)cccc12',
 'C=CC1(C)OC(=O)N(c2cc(Cl)cc(Cl)c2)C1=O',
 'c1ccc2c(c1)-c1cccc3c1c-2cc1ccccc13',
 'COc1cc(Cl)c(Cl)c(Cl)c1Cl',
 'CCCCCC(=O)O',
 'CCC#N',
 'Brc1ccc(Br)c(Br)c1',
 'CCCCCCC=O',
 'CC(=O)N(C)C',
 'O=[N+]([O-])c1ccc([N+](=O)[O-])cc1',
 'Cc1cccc(C)c1C',
 'Clc1ccc(Cl)c(-c2c(Cl)ccc(Cl)c2Cl)c1Cl',
 'ClC(Br)Br',
 'Clc1ccc(Cl)c(Cl)c1Cl',
 'Clc1ccc(Cl)c(Cl)c1Cl',
 'CC(C)C(C(=O)OC(C#N)c1cccc(Oc2ccccc2)c1)c1ccc(OC(F)F)cc1',
 'CC1CCCC1',
 'CN(C)C',
 'FC(F)(F)c1ccccc1',
 'Nc1ccccc1N',
 'Clc1c(Cl)c(Cl)c(-c2ccccc2)c(Cl)c1Cl',
 'CCN(CC)c1ccccc1',
 'Oc1cccc(O)c1',
 'CCCCCCCCCC(=O)O',
 'CC(O)C(F)(F)F',
 'Clc1ccc2c(c1)Oc1ccccc1O2',
 'CC(=O)OCC(C)C',
 'CCCCCCCCCCCCCCCCCCO',
 'CC1CO1',
 'Cc1cccc(C(=O)O)c1',
 'CCOP(=O)(OCC)SCCSCC',
 'CCCN(CCC)C(=O)SCC',
 'c1ccc2c(c1)-c1cccc3cccc-2c13',
 'c1ccc2c(c1)-c1cccc3cccc-2c13',
 'ClC1(Cl)C2(Cl)C3(Cl)C4(Cl)C(Cl)(Cl)C5(Cl)C3(Cl)C1(Cl)C5(Cl)C24Cl',
 'Cc1ccc(C)c(C)c1',
 'c1cc2ccc3ccc4ccc5cccc6c(c1)c2c3c4c56',
 'CCCCCCCCC=O',
 'CC1=C(C(=O)Nc2ccccc2)SCCO1',
 'Oc1ccc(Br)cc1',
 'CBr',
 'CBr',
 'COc1cc(Cl)c(Cl)cc1O',
 'c1ccc2cc3c(cc2c1)-c1cccc2cccc-3c12',
 'c1ccc2c(c1)ccc1ncccc12',
 'CCCC(=O)CCC',
 'Clc1ccc(Cl)c(-c2cc(Cl)ccc2Cl)c1',
 'CNN',
 'ClC(Cl)Cl',
 'CC(C)Cc1ccc(C(C)C(=O)O)cc1',
 'CC(C)Cc1ccc(C(C)C(=O)O)cc1',
 'CCSCc1ccccc1OC(=O)NC',
 'Nc1ccc(-c2ccc(N)cc2)cc1',
 'COc1cccc(Cl)c1Cl',
 'Cc1ccc(C(C)C)cc1',
 'Oc1ccc(Cl)cc1Cl',
 'CCCCC=O',
 'C1=CCCCC1',
 'C=CC=O',
 'C=CC=O',
 'C=CC=O',
 'Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2',
 'O=[N+]([O-])c1ccc(Oc2ccc(C(F)(F)F)cc2[N+](=O)[O-])cc1',
 'CC(C)CCC(C)(C)C',
 'CC1=CC(=O)CC(C)(C)C1',
 'CCOC(=O)NCCOc1ccc(Oc2ccccc2)cc1',
 'BrC(Br)(Br)Br',
 'c1ccc2c(c1)Cc1c-2ccc2ccccc12',
 'CCCCOCN(C(=O)CCl)c1c(CC)cccc1CC',
 'CCCCOCN(C(=O)CCl)c1c(CC)cccc1CC',
 'Clc1cc(-c2ccccc2)c(Cl)c(Cl)c1Cl',
 'CC(C)CBr',
 'CCCCCCCCCCCCCCO',
 'CCCCCCCCCCCCCCO',
 'CCCCCCCCCCCCCCCCCCCCCCCCCC',
 'CCCCC(=O)OC',
 'O=C1CCCN1',
 'CN(C)CCN(Cc1cccs1)c1ccccn1',
 'CSc1ccccc1',
 'CCC(C)(C)C',
 'Oc1c(Cl)cccc1Cl',
 'CCOP(=S)(NC(C)C)Oc1ccccc1C(=O)OC(C)C',
 'Oc1cccc(Cl)c1',
 'CCCCCC1CCCC1',
 'OC1CCCCC1',
 'Cc1cccnc1C',
 'COP(=S)(OC)Oc1nc(Cl)c(Cl)cc1Cl',
 'C=CC(N)=O',
 'C=CC(N)=O',
 'Clc1cc(Cl)c2c(c1Cl)Oc1ccccc1O2',
 'Clc1cc(Cl)c2c(c1Cl)Oc1ccccc1O2',
 'CCCl',
 'C#C',
 'CCCOCC(=Nc1ccc(Cl)cc1C(F)(F)F)n1ccnc1',
 'Oc1ccc(Cl)c(Cl)c1Cl',
 'Nc1cc(Cl)cc(C(=O)O)c1Cl',
 'CCCOC(=O)CCC',
 'O=Cc1ccccc1',
 'Nc1ccc(-c2ccc(N)c(Cl)c2)cc1Cl',
 'CCI',
 'Cc1cncc(C)c1',
 'C1=Cc2cccc3cccc1c23',
 'CC(C)C(=O)C(C)C',
 'O=Cc1ccco1',
 'CCc1ccccc1C',
 'COc1ccc(C(c2ccc(OC)cc2)C(Cl)(Cl)Cl)cc1',
 'COc1ccc(C(c2ccc(OC)cc2)C(Cl)(Cl)Cl)cc1',
 'COc1ccc(C(c2ccc(OC)cc2)C(Cl)(Cl)Cl)cc1',
 'Cc1ccccc1Cl',
 'C#CCCC',
 'ClCCBr',
 'CCCCC(=O)OCC',
 'Clc1cc(Cl)c(-c2c(Cl)cc(Cl)cc2Cl)c(Cl)c1',
 'N=C(N)N[N+](=O)[O-]',
 'COc1c(Cl)cc(Cl)c(OC)c1O',
 'COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1',
 'COC(=O)Nc1cccc(OC(=O)Nc2cccc(C)c2)c1',
 'c1ccc2cc3cc4ccccc4cc3cc2c1',
 'Clc1ccc(-c2c(Cl)cccc2Cl)cc1',
 'CCC1CCCCC1',
 'CC(C)(c1ccc(O)cc1)c1ccc(O)cc1',
 'CCCC',
 'CCCC',
 'CC(C)CCCCCOC(=O)c1ccccc1C(=O)OCCCCCC(C)C',
 'CCCCCCCCC(=O)OC',
 'CC(C)C(C)(C)O',
 'CCCCN(CCCC)CCCC',
 'CNC(=O)Oc1ccc(N(C)C)c(C)c1',
 'CCCCCCC',
 'CI',
 'CCCCCCCCN',
 'Clc1ccccc1Oc1ccccc1',
 'O=[N+]([O-])c1ccccc1O',
 'CCOC1Oc2ccc(OS(C)(=O)=O)cc2C1(C)C',
 'CCCCOC(=O)c1ccccc1C(=O)OCc1ccccc1',
 'CCCCOC(=O)c1ccccc1C(=O)OCc1ccccc1',
 'Clc1cccc(Cl)c1Oc1ccccc1',
 'CCNc1nc(NC(C)(C)C)nc(SC)n1',
 'COc1nc(NC(C)C)nc(NC(C)C)n1',
 'Oc1ccccc1Cl',
 'CNC(=O)Oc1ccccc1OC(C)C',
 'CCC(O)C(C)C',
 'Oc1ccccc1',
 'COc1c(Cl)cc(Cl)c(Cl)c1Cl',
 'Clc1ccc(-c2cc(Cl)c(Cl)cc2Cl)cc1Cl',
 'Cc1cccc(C)c1',
 'CN(C)C(=O)Nc1ccc(Oc2ccc(Cl)cc2)cc1',
 'Clc1ccc2oc3ccc(Cl)cc3c2c1',
 'Cc1ccc(O)cc1',
 'COc1c(Cl)ccc(Cl)c1C(=O)O',
 'CCc1ccccc1O',
 'c1ccc2c(c1)sc1ccccc12',
 'c1ccc2c(c1)sc1ccccc12',
 'Clc1cccc(Cl)c1-c1ccccc1',
 'O=[N+]([O-])c1ccccc1[N+](=O)[O-]',
 'Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl',
 'Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl',
 'CCCCCC=O',
 'Clc1ccccc1-c1ccccc1Cl',
 'CN(C)c1ccc(N=Nc2ccccc2)cc1',
 'Clc1cc2c(oc3cc(Cl)c(Cl)c(Cl)c32)c(Cl)c1Cl',
 'O=C(O)CCCC(=O)O',
 'N#Cc1ccccc1',
 'Clc1cc(-c2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c(Cl)c1Cl',
 'Cc1occc1C(=O)Nc1ccccc1',
 'CCCCCCCC(=O)OC',
 'Clc1cc(Cl)c(Cl)cc1Cl',
 'Clc1cc(Cl)c(Cl)cc1Cl',
 'O=C1CCCCC1',
 'COP(=S)(OC)Oc1cc(Cl)c(Cl)cc1Cl',
 'O=C1C2CC=CCC2C(=O)N1SC(Cl)(Cl)C(Cl)Cl',
 'C=CC(=O)OCC',
 'Cc1cccc(N)c1',
 'CN(C)c1ccccc1',
 'ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl',
 'ClC1C(Cl)C(Cl)C(Cl)C(Cl)C1Cl',
 'CC(=O)OC(C)C',
 'CCOC(N)=O',
 'Clc1cc(Cl)c(Cl)c(-c2c(Cl)c(Cl)cc(Cl)c2Cl)c1Cl',
 'ClCBr',
 'CCCCCCCOC(=O)c1ccccc1C(=O)OCCCCCCC',
 'CCCCNCCCC',
 'CCc1ccc(C)cc1',
 'C=CCCCCCCC',
 'Brc1cccc(Br)c1',
 'CCC[N+](=O)[O-]',
 'Cc1c2ccccc2c(C)c2ccccc12',
 'Clc1cccc2ccccc12',
 'Clc1cc(Cl)c(-c2cccc(Cl)c2Cl)cc1Cl',
 'c1ccncc1',
 'CCCCCC(C)C',
 'Oc1ccc2ccccc2c1',
 'Clc1ccc(C(c2ccccc2Cl)C(Cl)Cl)cc1',
 'CC1CCCO1',
 'Cc1ccc(NC(=O)N(C)C)cc1Cl',
 'CCOC(=O)CC(SP(=S)(OC)OC)C(=O)OCC',
 'CCOC(=O)CC(SP(=S)(OC)OC)C(=O)OCC',
 'CCOC(=O)CC(SP(=S)(OC)OC)C(=O)OCC',
 'CC1CCCCC1NC(=O)Nc1ccccc1',
 'CCCC(C)O',
 'Cc1ccnc(C)c1',
 'CC1(C)C(C=C(Cl)Cl)C1C(=O)OC(C#N)c1cccc(Oc2ccccc2)c1',
 'c1cc[nH]c1',
 'C=CC=C',
 'CCCCCCCCO',
 'ClCc1ccccc1',
 'CCCCCCOC(=O)c1ccccc1C(=O)OCCCCCC',
 'CCCCCCOC(=O)c1ccccc1C(=O)OCCCCCC',
 'CCCC1CCCC1',
 'C#CCC',
 'O=C(O)CCC(=O)O',
 'Clc1ccc(Cl)c(-c2ccc(Cl)c(Cl)c2)c1',
 'Oc1c(Cl)c(Cl)cc(Cl)c1Cl',
 'CCCOC=O',
 'CC(C)O',
 'CC1(C)CON(Cc2ccccc2Cl)C1=O',
 'Oc1cccc2ccccc12',
 'Oc1cccc2ccccc12',
 'Clc1ccc(Cl)c(-c2ccccc2)c1',
 'Nc1nc[nH]n1',
 'O=[N+]([O-])c1cccc(Cl)c1',
 'CCCCI',
 'Cc1ccc(CO)cc1',
 'CC(Cl)CCl',
 'CC(C)(C)CO',
 'Clc1cc(Cl)c(-c2c(Cl)cc(Cl)c(Cl)c2Cl)cc1Cl',
 'CC(C)C(=O)O',
 'CCC(=O)c1ccccc1',
 'Clc1cc2oc3cc(Cl)c(Cl)cc3c2cc1Cl',
 'Clc1cccc(-c2cccc(Cl)c2)c1',
 'CN(C)C(=S)SSC(=S)N(C)C',
 'Cc1ccc(O)c(C)c1',
 'CCc1ccccc1CC',
 'CC1CCCCC1C',
 'C=CCCCCCC',
 'CC(=O)c1ccccc1',
 'NC(=O)c1ccccc1',
 'COC(C)=O',
 'COC(C)=O',
 'CC(C)Cc1ccccc1',
 'COc1c(Cl)cccc1Cl',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2Cl)c(Cl)c1',
 'Clc1ccc(Cl)c(-c2cc(Cl)c(Cl)cc2Cl)c1',
 'Clc1ccc(Cl)c(-c2cc(Cl)c(Cl)cc2Cl)c1',
 'CCCCC(C)CC',
 'Clc1ccccc1-c1ccccc1',
 'CCOP(=S)(OCC)Oc1ccc([N+](=O)[O-])cc1',
 'CCOP(=S)(OCC)Oc1ccc([N+](=O)[O-])cc1',
 'CCOP(=S)(OCC)Oc1ccc([N+](=O)[O-])cc1',
 'Clc1ccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)cc1Cl',
 'O=[N+]([O-])C(Cl)(Cl)Cl',
 'CCO',
 'CCC(=O)Nc1ccc(Cl)c(Cl)c1',
 'CCC(=O)Nc1ccc(Cl)c(Cl)c1',
 'COP(=S)(OC)Oc1cc(Cl)c(I)cc1Cl',
 'CN=C=S',
 'O=C1OC(=O)c2ccccc21',
 'C=C(Cl)Cl',
 'CCCCC(C=O)CC',
 'COc1ccccc1O',
 'O=C1C2(Cl)C3(Cl)C4(Cl)C(Cl)(Cl)C5(Cl)C3(Cl)C1(Cl)C5(Cl)C24Cl',
 'Oc1cc(Cl)c(Cl)cc1Cl',
 'c1ccc2c(c1)ccc1cc3c(ccc4ccccc43)cc12',
 'C=CCCCC',
 'Oc1c(Cl)cc(Cl)cc1Cl',
 'O=C(Nc1ccccc1)c1ccccc1I',
 'CCSCC',
 'Clc1cc(Cl)c(-c2c(Cl)ccc(Cl)c2Cl)cc1Cl',
 'CON(C)C(=O)Nc1ccc(Cl)cc1',
 'O=[N+]([O-])c1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl',
 'Oc1ccccc1-c1ccccc1',
 'CCCCCCCI',
 'COP(=S)(OC)SCN1C(=O)c2ccccc2C1=O',
 'C1=CCCC1',
 'O=[N+]([O-])c1ccccc1',
 'O=[N+]([O-])c1ccccc1',
 'CCCCCCCCCCCCO',
 'CCCCO',
 'Cn1c(=O)on(-c2ccc(Cl)c(Cl)c2)c1=O',
 'CCCCCCCBr',
 'CCCCCCc1ccccc1',
 'Clc1ccc(Cl)c(-c2ccccc2)c1Cl',
 'Clc1cc2oc3c(Cl)c(Cl)c(Cl)c(Cl)c3c2cc1Cl',
 'OCCO',
 'C[N+](=O)[O-]',
 'CCOCCOCC',
 'Oc1cccc2cccnc12',
 'O=C(Nc1cccc(Cl)c1)OCC#CCCl',
 'CC[N+](=O)[O-]',
 'CCCOC(=O)CC',
 'CNC(=O)CSP(=S)(OC)OC',
 'Oc1ccc(Cl)cc1Cc1cc(Cl)ccc1O',
 'CC(C)N(C(=O)CCl)c1ccccc1',
 'CCNc1nc(NCC)nc(SC)n1',
 'Cc1ccc(C(C)C)c(O)c1',
 'Cc1ccc(C(C)C)c(O)c1',
 'CC(C)COC(=O)c1ccccc1C(=O)OCC(C)C',
 'CC(C)COC(=O)c1ccccc1C(=O)OCC(C)C',
 'C=C(C)CC',
 'O=[N+]([O-])OCCCCCO[N+](=O)[O-]',
 'CC(=O)OCCC(C)C',
 'CCCCCCCl',
 'CCCCCCBr',
 'CCCCBr',
 'CCCOC(=O)c1ccccc1C(=O)OCCC',
 'CCCCCCCCOC(=O)c1ccccc1C(=O)OCCCCCCCC',
 'CCC(C)(C)O',
 'Clc1cc(Cl)c(-c2c(Cl)cccc2Cl)c(Cl)c1',
 'Clc1cc(Cl)c(-c2ccccc2)c(Cl)c1',
 'CCOC(=O)c1ccccc1',
 'O=C(O)C(=O)O',
 'CCOP(=S)(OCC)SC(CCl)N1C(=O)c2ccccc2C1=O',
 'CCOP(=S)(OCC)SC(CCl)N1C(=O)c2ccccc2C1=O',
 'Clc1ccccc1',
 'Clc1ccccc1',
 'Clc1cc(Cl)c(-c2cc(Cl)c(Cl)cc2Cl)cc1Cl',
 'CC(C)N(C(=O)SCC(Cl)=C(Cl)Cl)C(C)C',
 'CCCCCCCCCCO',
 'c1ccc2cc3ccccc3cc2c1',
 'c1ccc2cc3ccccc3cc2c1',
 'C#CC',
 'CCNc1nc(Cl)nc(NC(C)C)n1',
 'CCNc1nc(Cl)nc(NC(C)C)n1',
 'CC(Oc1ccc(Cl)cc1Cl)C(=O)O',
 'CCNc1nc(Cl)nc(NC(C)(C)C#N)n1',
 'CC(C)Br',
 'CCC=O',
 'Clc1ccc2c(c1)Oc1ccc(Cl)cc1O2',
 'Cc1[nH]c(=O)n(C(C)(C)C)c(=O)c1Cl',
 'COc1c(Cl)c(Cl)c(Cl)c(Cl)c1OC',
 'COc1c(Cl)c(Cl)c(Cl)c(Cl)c1OC',
 'CCl',
 'CCCN(CCC)c1c([N+](=O)[O-])cc(C(C)C)cc1[N+](=O)[O-]',
 'COP(=O)(OC)C(O)C(Cl)(Cl)Cl',
 'COP(=O)(OC)C(O)C(Cl)(Cl)Cl',
 'c1ccc2c(c1)Oc1ccccc1O2',
 'c1ccc2c(c1)Oc1ccccc1O2',
 'CC1CCC(C(C)C)C(=O)C1',
 'CCCC(C)Cl',
 'Clc1ccc(Br)cc1',
 'OC(c1ccc(Cl)cc1)(c1cncnc1)c1ccccc1Cl',
 'CC(C)CC(C)(C)C',
 'COc1cc(Cl)c(Cl)c(Cl)c1O',
 'c1ccc2c(c1)Cc1ccccc1-2',
 'c1ccc2c(c1)Cc1ccccc1-2',
 'COC(=O)c1ccccc1C(=O)OC',
 'COC(=O)c1ccccc1C(=O)OC',
 'COCOC',
 'Cc1ccc(C)c(O)c1',
 'C=CC#N',
 'CON(C)C(=O)Nc1ccc(Br)c(Cl)c1',
 'Clc1cccc(-c2ccccc2)c1',
 'C1CCC2CCCCC2C1',
 'CC(C)C',
 'Cc1ccc(Cl)cc1',
 'Nc1c(Cl)c(Cl)nc(C(=O)O)c1Cl',
 'C1CCNCC1',
 'CCc1ccccn1',
 'N#Cc1cc(I)c(O)c(I)c1',
 'O=[N+]([O-])c1ccc(Cl)c(Cl)c1',
 'CC(=O)CC(C)C',
 'CC(=O)CC(C)C',
 'CCCCCCCC=O',
 'CCCCCC(=O)OC',
 'Cc1ccncc1C',
 'CNc1cnn(-c2cccc(C(F)(F)F)c2)c(=O)c1Cl',
 'CCCOCC',
 'c1ccc(CNc2ncnc3[nH]cnc23)cc1',
 'Cc1cccc(C)c1O',
 'COP(=S)(OC)Oc1cc(Cl)c(Br)cc1Cl',
 'ICI',
 'CC1CCC(C)(C)C1',
 'CCCC(=O)OCC',
 'Clc1ccc(Cl)cc1',
 'Clc1ccc(Cl)cc1',
 'CCCCc1ccccc1',
 'CCNc1nc(Cl)nc(NCC)n1',
 'CCCc1ccccc1',
 'CCCCC',
 'CCN(CC)c1c([N+](=O)[O-])cc(C(F)(F)F)c(N)c1[N+](=O)[O-]',
 'Clc1cc2oc3c(Cl)c(Cl)c(Cl)cc3c2cc1Cl',
 'Cc1cccc(C)n1',
 'CCNc1ccccc1',
 'CO',
 'CCCSC(=O)N(CCC)CCC',
 'Clc1cccc(-c2cc(Cl)ccc2Cl)c1',
 'Clc1c(Cl)c(Cl)c(-c2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'CCSCCSP(=S)(OC)OC',
 'Clc1ccc(Cl)c(-c2ccc(Cl)c(Cl)c2Cl)c1',
 'C=CCCl',
 'Cc1cccc(Cl)c1',
 'C#CCCCCCCC',
 'COC(=O)c1ccccc1O',
 'COP(=S)(OC)Oc1ccc([N+](=O)[O-])cc1Cl',
 'CC',
 'Clc1cc(Cl)c2c(c1)Oc1c(Cl)cc(Cl)cc1O2',
 'O=C(O)COc1ccc(Cl)cc1Cl',
 'CCC(C)c1cc([N+](=O)[O-])cc([N+](=O)[O-])c1O',
 'BrCCBr',
 'Nc1ccccc1',
 'Clc1ccc(Cl)c(Cl)c1',
 'Clc1ccc(Cl)c(Cl)c1',
 'Clc1cc(Cl)c(Oc2cc(Cl)c(Cl)cc2Cl)cc1Cl',
 'OCC(O)CO',
 'CCCC(=O)CC',
 'CCC(C)CC',
 'COc1cc(Cl)c(OC)cc1Cl',
 'CCCC(C)CO',
 'CCCCCOC(C)=O',
 'Clc1cc(Cl)c(Oc2ccc(Cl)c(Cl)c2Cl)cc1Cl',
 'Clc1cc(Cl)c(Oc2ccc(Cl)c(Cl)c2Cl)cc1Cl',
 'CC(C)CCO',
 'CC(=O)C(C)(C)C',
 'Clc1ccc(Oc2ccc(Cl)c(Cl)c2Cl)cc1Cl',
 'Cc1ccccc1[N+](=O)[O-]',
 'Cc1ccccc1[N+](=O)[O-]',
 'Nc1ccc(O)cc1',
 'Nc1cccc(Cl)c1',
 'COP(=S)(OC)Oc1ccc(SC)c(C)c1',
 'CCCCOCCO',
 'CCCOCCC',
 'C=Cc1ccccc1',
 'C=Cc1ccccc1',
 'Cc1ccc2cc3c(ccc4ccccc43)c3c2c1CC3',
 'Cc1ccc2cc3c(ccc4ccccc43)c3c2c1CC3',
 'c1ccc2c(c1)CCC2',
 'c1ccc2c(c1)CCC2',
 'C=C(C)CN(CC)c1c([N+](=O)[O-])cc(C(F)(F)F)cc1[N+](=O)[O-]',
 'CC(C)c1ccc(NC(=O)N(C)C)cc1',
 'CC(C)c1ccc(NC(=O)N(C)C)cc1',
 'CCCCCc1ccccc1',
 'Clc1cccc(Cl)c1-c1c(Cl)cccc1Cl',
 'Clc1cccc(Cl)c1-c1c(Cl)cccc1Cl',
 'FC(F)(Cl)Cl',
 'CCNc1nc(NC(C)C)nc(OC)n1',
 'C=CCCC=C',
 'CCCCOC(=O)CCCCCCCCC(=O)OCCCC',
 'CCCCOC(=O)CCCCCCCCC(=O)OCCCC',
 'Fc1ccccc1',
 'Clc1ccc(Cl)c(-c2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c1',
 'CN(C)C(=O)C(c1ccccc1)c1ccccc1',
 'Brc1ccccc1',
 'CN1CSC(=S)N(C)C1',
 'ClC1=C(Cl)C2(Cl)C3C4C=CC(C4)C3C1(Cl)C2(Cl)Cl',
 'ClC1=C(Cl)C2(Cl)C3C4C=CC(C4)C3C1(Cl)C2(Cl)Cl',
 'OCCOc1ccccc1',
 'OCCOc1ccccc1',
 'Brc1cc(Br)c(Br)cc1Br',
 'OC1CCCCCC1',
 'O=[N+]([O-])OCCCCO[N+](=O)[O-]',
 'Clc1ccccc1-c1c(Cl)cc(Cl)c(Cl)c1Cl',
 'COc1ccccc1Cl',
 'Cc1cc(C)c2ccccc2c1',
 'CC(C)(C)c1ccc(O)cc1',
 'CCC(C)(C)c1ccccc1',
 'CCN(CC)C(=O)SCc1ccc(Cl)cc1',
 'Oc1ccc(O)cc1',
 'Cc1ccc2ccccc2c1',
 'CCCCCCCCCCCCCCCCCC(=O)O',
 'CSC',
 'Clc1ccc(-c2ccc(Cl)cc2Cl)c(Cl)c1',
 'ClC(Cl)C(Cl)Cl',
 'COC(=O)c1ccccc1',
 'CCC(C)CO',
 'Cc1cccc2ccccc12',
 'CC1(C)C(C=C(Cl)Cl)C1C(=O)OC(C#N)c1ccc(F)c(Oc2ccccc2)c1',
 'C=CC',
 'O=C(NC(=O)c1c(F)cccc1F)Nc1ccc(Cl)cc1',
 'O=C(NC(=O)c1c(F)cccc1F)Nc1ccc(Cl)cc1',
 'CCCCC(C)=O',
 'CC1CCC(C)CC1',
 'c1ccc2[nH]nnc2c1',
 'Nc1ccc(Cl)cc1',
 'CC(C)CCl',
 'CC(=O)O',
 'C=CCOC(=O)c1ccccc1C(=O)OCC=C',
 'C=CCOC(=O)c1ccccc1C(=O)OCC=C',
 'Fc1ccc(F)cc1',
 'ClCC(Cl)(Cl)Cl',
 'Cc1cccc(O)c1C',
 'CCC(C)C',
 'c1ccc(CCc2ccccc2)cc1',
 'Cc1ccccc1O',
 'Cc1ccc(N=CN(C)C=Nc2ccc(C)cc2C)c(C)c1',
 'Cc1ccc(N=CN(C)C=Nc2ccc(C)cc2C)c(C)c1',
 'CC12CC1(C)C(=O)N(c1cc(Cl)cc(Cl)c1)C2=O',
 'BrC(Br)C(Br)Br',
 'CCc1ccc(C(=O)NN(C(=O)c2cc(C)cc(C)c2)C(C)(C)C)cc1',
 'C=CCC=C',
 'Cc1cccc([N+](=O)[O-])c1',
 'Cc1cccc([N+](=O)[O-])c1',
 'Clc1cc(Cl)c(Cl)c(Cl)c1',
 'Clc1cc(Cl)c(Cl)c(Cl)c1',
 'CCBr',
 'COc1ccccc1',
 'O=S1OCC2C(CO1)C1(Cl)C(Cl)=C(Cl)C2(Cl)C1(Cl)Cl',
 'O=S1OCC2C(CO1)C1(Cl)C(Cl)=C(Cl)C2(Cl)C1(Cl)Cl',
 'Cc1ccc(C(C)C)cc1O',
 'CCCCCCCCCCCCCCCO',
 'Clc1cccc(Cl)c1Cl',
 'Clc1cccc(Cl)c1Cl',
 'Clc1ccccc1Cl',
 'Clc1ccccc1Cl',
 'O=Cc1ccc(O)cc1',
 'C1CCCC1',
 'CCCCCCCCCC(=O)OCC',
 'Clc1c(Cl)c(Cl)c2c(oc3c(Cl)c(Cl)c(Cl)c(Cl)c32)c1Cl',
 'Clc1ccc(-c2ccccc2Cl)cc1Cl',
 'CCCCNC(=O)n1c(NC(=O)OC)nc2ccccc21',
 'CCOP(=S)(OCC)SCSP(=S)(OCC)OCC',
 'O=[N+]([O-])c1cccc2ccccc12',
 'O=[N+]([O-])c1cccc2ccccc12',
 'O=C(Nc1ccc([N+](=O)[O-])cc1Cl)c1cc(Cl)ccc1O',
 'CCc1ccc2ccccc2c1',
 'CCC(=O)OC',
 'COc1c(O)cc(Cl)c(Cl)c1Cl',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2)c(Cl)c1',
 'COP(=S)(OC)Oc1ccc([N+](=O)[O-])cc1',
 'CC(C)C(C)C(C)C',
 'Cc1cccc(O)c1',
 'Cc1ccc([N+](=O)[O-])cc1[N+](=O)[O-]',
 'COC(C)C',
 'CC(Cl)C(C)Cl',
 'CC(C)OC=O',
 'Sc1ccccc1',
 'OC(O)C(Cl)(Cl)Cl',
 'Cc1ccccc1N',
 'Clc1cc(Cl)c(-c2ccc(Cl)c(Cl)c2Cl)cc1Cl',
 'CCNc1nc(NC(C)C)nc(SC)n1',
 'C=CCc1ccc(O)c(OC)c1',
 'Oc1c(O)c(Cl)c(Cl)c(Cl)c1Cl',
 'C=CC(=C)C',
 'CC(C)OP(=S)(OC(C)C)SCCNS(=O)(=O)c1ccccc1',
 'O=CNC(N1CCN(C(NC=O)C(Cl)(Cl)Cl)CC1)C(Cl)(Cl)Cl',
 'Clc1ccc(Oc2ccc(Cl)c(Cl)c2Cl)c(Cl)c1',
 'CCOCC',
 'O=[N+]([O-])c1cccc(O)c1',
 'Nc1cccc2ccccc12',
 'Oc1cc(Cl)c(Cl)c(Cl)c1Cl',
 'CC(O[N+](=O)[O-])C(C)O[N+](=O)[O-]',
 'Clc1cccc(-c2c(Cl)c(Cl)cc(Cl)c2Cl)c1Cl',
 'Clc1cccc(-c2c(Cl)c(Cl)cc(Cl)c2Cl)c1Cl',
 'O=C(c1ccccc1)c1ccccc1',
 'CC(C)c1ccccc1',
 'ClC(Cl)=C(Cl)C(Cl)=C(Cl)Cl',
 'ClC(Cl)=C(Cl)C(Cl)=C(Cl)Cl',
 'Cn1c(=O)c2c(ncn2C)n(C)c1=O',
 'Clc1ccc(-c2c(Cl)cc(Cl)cc2Cl)cc1',
 'CN(C)C(=O)Nc1cccc(C(F)(F)F)c1',
 'O=C1c2ccccc2C(=O)c2ccccc21',
 'Cc1ccc(N)cc1',
 'CCCCC(CC)COC(=O)CCCCC(=O)OCC(CC)CCCC',
 'O=Cc1ccc2c(c1)OCO2',
 'CC(C)=CC1C(C(=O)OCc2cccc(Oc3ccccc3)c2)C1(C)C',
 'CCOP(=S)(OCC)Oc1nc(Cl)c(Cl)cc1Cl',
 'CCOP(=S)(OCC)Oc1nc(Cl)c(Cl)cc1Cl',
 'COCCOCCOC',
 'CC(Cl)Cl',
 'Cc1cccc2c(C)cccc12',
 'C=C(C)C(=C)C',
 'CNC(=O)Oc1ccccc1C1OCCO1',
 'Nc1ccc(Cl)c(Cl)c1',
 'CN1CCCC1=O',
 'CCCC(Cl)Cl',
 'Clc1ccc(Cl)c(-c2c(Cl)cccc2Cl)c1',
 'CCCCN(CC)C(=O)SCCC',
 'CCCCN(CC)C(=O)SCCC',
 'CCCCCC(=O)OCC',
 'c1ccc2[nH]ccc2c1',
 'CCCCCCC(C)O',
 'Clc1ccc2c(c1)Oc1cc(Cl)c(Cl)c(Cl)c1O2',
 'c1ccc2c(c1)c1ccccc1c1ccccc21',
 'CNC(=O)Oc1cc(C(C)(C)C)cc(C(C)(C)C)c1',
 'CCCCN(CC)c1c([N+](=O)[O-])cc(C(F)(F)F)cc1[N+](=O)[O-]',
 'c1ccc2ncccc2c1',
 'O=C(O)COc1cc(Cl)c(Cl)cc1Cl',
 'O=C(O)COc1cc(Cl)c(Cl)cc1Cl',
 'Ic1ccccc1',
 'Ic1ccccc1',
 'O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-]',
 'CCCCl',
 'Cc1c([N+](=O)[O-])cc([N+](=O)[O-])cc1[N+](=O)[O-]',
 'CCCCCCl',
 'Cc1cc(C)c(C)cc1C',
 'CCCCOP(=O)(OCCCC)OCCCC',
 'CCCC1COC(Cn2cncn2)(c2ccc(Cl)cc2Cl)O1',
 'Cc1cc(Cl)ccc1N=CN(C)C',
 'COC(=O)c1cc(Oc2ccc(Cl)cc2Cl)ccc1[N+](=O)[O-]',
 'Brc1ccc(Br)cc1',
 'CN1CC(O)N(c2nnc(C(C)(C)C)s2)C1=O',
 'CC(=O)C(C)C',
 'Cc1cc(C)cc(O)c1',
 'CCCCCCCC',
 'CCCCCCC(=O)OCC',
 'CCCCCOC(=O)CC',
 'c1ccc2c(c1)oc1ccccc12',
 'c1ccc2c(c1)oc1ccccc12',
 'c1ccc2c(c1)c1cccc3ccc4cccc2c4c31',
 'Nc1ccc([N+](=O)[O-])cc1',
 'COC=O',
 'Clc1ccc(-c2c(Cl)cc(Cl)c(Cl)c2Cl)c(Cl)c1Cl',
 'CC1CCCCC1',
 'O=[N+]([O-])c1cc([N+](=O)[O-])cc([N+](=O)[O-])c1',
 'ClCC(Cl)Cl',
 'O=NN(c1ccccc1)c1ccccc1',
 'Clc1ccc(Cl)c(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c1',
 'ClC(Cl)=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1',
 'Clc1cc(Cl)c(-c2c(Cl)c(Cl)cc(Cl)c2Cl)cc1Cl',
 'Fc1cccc(F)c1',
 'Clc1ccc(-c2ccccc2Cl)cc1',
 'c1ccc2ccccc2c1',
 'Clc1ccc(Cl)c(-c2c(Cl)c(Cl)cc(Cl)c2Cl)c1',
 'c1ccccc1',
 'CCC(C)C(C)=O',
 'c1ccc2c(c1)[nH]c1ccccc12',
 'C#CC(C)OC(=O)Nc1cccc(Cl)c1',
 'CC(C)Cl',
 'NC(N)=S',
 'C#CCCCCCC',
 'Oc1cccc(Cl)c1Cl',
 'CCCN(CCC)c1c([N+](=O)[O-])cc(S(N)(=O)=O)cc1[N+](=O)[O-]',
 'OCC(O)C(O)CO',
 'OC(c1ccc(Cl)cc1)(c1ccc(Cl)cc1)C(Cl)(Cl)Cl',
 'CC(Cl)(Cl)Cl',
 'N#Cc1cc(Br)c(O)c(Br)c1',
 'CCCCCCCCCCCC(=O)OC',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2)cc1Cl',
 'CC(=O)OCC(COC(C)=O)OC(C)=O',
 'Cc1ccc(C)cc1',
 'c1ccc(Oc2ccccc2)cc1',
 'c1ccc(Oc2ccccc2)cc1',
 'Clc1cc(Cl)cc(-c2cc(Cl)cc(Cl)c2)c1',
 'CCCC(=O)OC',
 'C1=CCC=CC1',
 'O=NN1CCCCC1',
 'Clc1ccc(-c2cc(Cl)ccc2Cl)cc1',
 'OCCCc1ccccc1',
 'CCOP(=S)(OCC)SCSc1ccc(Cl)cc1',
 'CCOP(=S)(OCC)SCSc1ccc(Cl)cc1',
 'BrCCCBr',
 'CCCCCl',
 'C=C',
 'Clc1cc(Cl)c(-c2ccccc2)cc1Cl',
 'Oc1c(Cl)cc(Cl)c(Cl)c1Cl',
 'Cc1nnc(-c2ccccc2)c(=O)n1N',
 'ClC(Cl)(Cl)C(Cl)(Cl)Cl',
 'CCOC(=O)CCCCC(=O)OCC',
 'CCCCCCCC(C)=O',
 'COc1cc(Cl)c(Cl)c(Cl)c1OC',
 'CCCCCC',
 'CCCCCCCCBr',
 'CCCOC',
 'CCCOC',
 'C',
 'CNC(=O)Oc1cccc(C)c1',
 'Oc1cc(Cl)cc(Cl)c1',
 'Clc1cccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c1Cl',
 'Clc1cccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)c1Cl',
 'C=C(C)C',
 'Clc1cccc(-c2cccc(Cl)c2Cl)c1Cl',
 'ClC1=C(Cl)C(Cl)(Cl)C(Cl)=C1Cl',
 'C=C(C)C(=O)OC',
 'C=CCC',
 'ClC(Cl)(Cl)Cl',
 'ClC(Cl)(Cl)Cl',
 'ClCC(Br)CBr',
 'CCOC=O',
 'Clc1cc2c(c(Cl)c1Cl)Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1O2',
 'COc1c(Cl)cc(Cl)cc1Cl',
 'Clc1cc2c(cc1Cl)Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1O2',
 'O=[N+]([O-])c1cccc([N+](=O)[O-])c1',
 'c1ccc2c(c1)ccc1c3ccccc3ccc21',
 'Clc1cc2c(oc3c(Cl)c(Cl)c(Cl)c(Cl)c32)c(Cl)c1Cl',
 'CC12CCC(CC1)C(C)(C)O2',
 'CCCNCCC',
 'Clc1ccc(-c2ccccc2)c(Cl)c1',
 'CC#N',
 'CCCC(C)(C)O',
 'CCC(C)C(C)C',
 'Cc1cc(Cl)ccc1OCC(=O)O',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2Cl)cc1Cl',
 'Cc1cccc(Br)c1',
 'CC(C)Nc1nc(Cl)nc(NC(C)C)n1',
 'C1CCCCCCC1',
 'CCCCCCCN',
 'O=C(O)Cc1ccccc1',
 'Clc1ccc2c(c1)Oc1cc(Cl)ccc1O2',
 'O=C(O)CCCCC(=O)O',
 'Cc1cc(Cl)ccc1OC(C)C(=O)O',
 'COC(=O)c1ccc(C(=O)OC)cc1',
 'COC(=O)c1ccc(C(=O)OC)cc1',
 'O=C(O)c1ccccc1',
 'CCCCS',
 'Brc1ccccc1Br',
 'CSSC',
 'OCCc1ccccc1',
 'Clc1ccc2c(c1)Oc1c(Cl)c(Cl)c(Cl)c(Cl)c1O2',
 'CCCCC(=O)CCCC',
 'ClCCCl',
 'C=CCC(C)C',
 'CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1cncn1',
 'OCc1ccccc1',
 'CCc1cccc(C)c1N(C(=O)CCl)C(C)COC',
 'CC(C)CC(C)O',
 'Oc1cc(Cl)cc(Cl)c1Cl',
 'Cc1cc2ccccc2cc1C',
 'Cc1ccccc1C(C)C',
 'C=C(C)CCC',
 'CCCCCCCC(=O)O',
 'Oc1ccc(Cl)cc1',
 'OCC(CO)(CO)CO',
 'CCN(CC)C(=O)C(C)Oc1cccc2ccccc12',
 'Nc1cnn(-c2ccccc2)c(=O)c1Cl',
 'Nc1cnn(-c2ccccc2)c(=O)c1Cl',
 'CCOP(=O)(OCC)Oc1ccc([N+](=O)[O-])cc1',
 'C1CCOC1',
 'Clc1cc(Cl)c(-c2ccccc2Cl)cc1Cl',
 'CCc1cccc(CC)c1N(COC)C(=O)CCl',
 'CCc1cccc(CC)c1N(COC)C(=O)CCl',
 'CCc1cccc(CC)c1N(COC)C(=O)CCl',
 'Clc1cc(Cl)c(Oc2ccc(Cl)c(Cl)c2Cl)c(Cl)c1',
 'C1CCCCC1',
 'CCOC(C)=O',
 'CC(Br)CBr',
 'Clc1cc(Cl)c(Cl)c(-c2c(Cl)c(Cl)c(Cl)c(Cl)c2Cl)c1Cl',
 'O=[N+]([O-])c1ccc(Oc2ccc(Cl)cc2Cl)cc1',
 'C=C(Cl)CSC(=S)N(CC)CC',
 'Nc1cccc([N+](=O)[O-])c1',
 'Clc1cc(Cl)cc(Cl)c1',
 'Clc1cc(Cl)cc(Cl)c1',
 'CON(C)C(=O)Nc1ccc(Cl)c(Cl)c1',
 'Clc1cc(Cl)c(Oc2cc(Cl)c(Cl)cc2Cl)c(Cl)c1',
 'CC(C)CCOC=O',
 'CCOP(=S)(OCC)SCSC(C)(C)C',
 'CCOP(=S)(OCC)SCSC(C)(C)C',
 'C=C(C)C1CC=C(C)CC1',
 'CN(C)C(=O)Nc1ccc(Cl)c(Cl)c1',
 'CN(C)C(=O)Nc1ccc(Cl)c(Cl)c1',
 'O=C(O)c1ccccc1C(=O)O',
 'O=C(O)c1ccccc1C(=O)O',
 'CCCC(C)=O',
 'CC1(C)C(C(=O)OC(C#N)c2cccc(Oc3ccccc3)c2)C1(C)C',
 'CCOP(=S)(OCC)Oc1nc(Cl)n(C(C)C)n1',
 'NC(N)=O',
 'ClC=C(Cl)Cl',
 'Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-]',
 'Cc1ccc(C)nc1',
 'CN(C)C(=O)Nc1ccccc1',
 'CC(C)SC(C)C',
 'COP(=O)(NC(C)=O)SC',
 'c1ccc(NNc2ccccc2)cc1',
 'Nc1ccc(Cc2ccc(N)cc2)cc1',
 'Clc1cc(Cl)c(Oc2cc(Cl)c(Cl)c(Cl)c2Cl)cc1Cl',
 'Oc1ccccc1O',
 'Cc1ccc(O)cc1C',
 'O=C1CCCO1',
 'CNC(=O)Oc1cc(C)c(SC)c(C)c1',
 'Cc1nc(N(C)C)nc(OC(=O)N(C)C)c1C',
 'CCCN(CCC)CCC',
 'O=C(O)c1ccccc1O',
 'Cc1ccc2cc(C)ccc2c1',
 'C=CC(C)C',
 'ClCCCCl',
 'CCCCC(CC)CO',
 'CCCOC(C)=O',
 'c1ccoc1',
 'ClCCl',
 'C1COCCN1',
 'c1ccc2c(c1)ccc1ccccc12',
 'CCc1ccccc1',
 'N#Cc1c(Cl)cccc1Cl',
 'COc1ccc(Cl)cc1',
 'CCCCCCCCCC(=O)OC',
 'CC(=O)CC(C)=O',
 'Clc1ccc(-c2ccc(Cl)cc2)cc1',
 'c1ccsc1',
 'CC(C)(C)c1ccccc1',
 'COc1cccc(Cl)c1',
 'Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1',
 'Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1',
 'C#CCCCCC',
 'C=CCCC',
 'Cn1cc(-c2ccccc2)c(=O)c(-c2cccc(C(F)(F)F)c2)c1',
 'Cc1ccccc1',
 'Oc1ccc(-c2ccccc2)cc1',
 'CC(C)CCBr',
 'CCCCCCCC(=O)OCC',
 'Clc1ccc(-c2ccc(Cl)c(Cl)c2)cc1',
 'CCCCCBr',
 'COc1c(O)c(Cl)c(Cl)c(Cl)c1Cl',
 'CC(C)OC(C)C',
 'CCCCCCCCC(C)=O',
 'Cc1ccc(Br)cc1',
 'ClC(Cl)=C(c1ccc(Cl)cc1)c1ccccc1Cl',
 'CCCSCCC',
    ]):
        m = Chem.MolFromSmiles(smiles)
        fgs = identify_functional_groups(m)
        print ('%2d: %d fgs'%(ix+1, len(fgs)), fgs)


if __name__ == "__main__":
    main() 

 1: 6 fgs [IFG(atomIds=(1,), atoms='S', type='cSC'), IFG(atomIds=(3,), atoms='n', type='cnc'), IFG(atomIds=(5,), atoms='N', type='cNC'), IFG(atomIds=(9,), atoms='n', type='cnc'), IFG(atomIds=(11,), atoms='N', type='cNC'), IFG(atomIds=(15,), atoms='n', type='cnc')]
 2: 0 fgs []
 3: 0 fgs []
 4: 1 fgs [IFG(atomIds=(1,), atoms='N', type='cNC')]
 5: 1 fgs [IFG(atomIds=(0, 1), atoms='C=C', type='C=CC')]
 6: 4 fgs [IFG(atomIds=(0,), atoms='Cl', type='cCl'), IFG(atomIds=(10,), atoms='Cl', type='cCl'), IFG(atomIds=(14,), atoms='Cl', type='CCl'), IFG(atomIds=(15,), atoms='Cl', type='CCl')]
 7: 2 fgs [IFG(atomIds=(1, 2, 3, 4, 6), atoms='OP(O)(=S)S', type='COP(=S)(OC)SC'), IFG(atomIds=(8, 9, 10, 12, 13), atoms='O=CNC=O', type='CC(=O)N(C)C=O')]
 8: 5 fgs [IFG(atomIds=(0,), atoms='Cl', type='cCl'), IFG(atomIds=(10,), atoms='Cl', type='cCl'), IFG(atomIds=(12,), atoms='Cl', type='cCl'), IFG(atomIds=(14,), atoms='Cl', type='cCl'), IFG(atomIds=(16,), atoms='Cl', type='cCl')]
 9: 6 fgs [IFG(atomIds=(0,)

282: 1 fgs [IFG(atomIds=(0, 1, 5), atoms='NC=O', type='CNC(C)=O')]
283: 4 fgs [IFG(atomIds=(1,), atoms='N', type='CN(C)C'), IFG(atomIds=(11,), atoms='s', type='csc'), IFG(atomIds=(5,), atoms='N', type='cN(C)C'), IFG(atomIds=(17,), atoms='n', type='cnc')]
284: 1 fgs [IFG(atomIds=(1,), atoms='S', type='cSC')]
285: 0 fgs []
286: 3 fgs [IFG(atomIds=(0,), atoms='O', type='cO'), IFG(atomIds=(8,), atoms='Cl', type='cCl'), IFG(atomIds=(3,), atoms='Cl', type='cCl')]
287: 2 fgs [IFG(atomIds=(2, 3, 4, 5, 9), atoms='NP(O)(O)=S', type='cOP(=S)(NC)OC'), IFG(atomIds=(16, 17, 18), atoms='O=CO', type='cC(=O)OC')]
288: 2 fgs [IFG(atomIds=(0,), atoms='O', type='cO'), IFG(atomIds=(6,), atoms='Cl', type='cCl')]
289: 0 fgs []
290: 1 fgs [IFG(atomIds=(0,), atoms='O', type='CO')]
291: 1 fgs [IFG(atomIds=(5,), atoms='n', type='cnc')]
292: 5 fgs [IFG(atomIds=(1, 2, 3, 4, 6), atoms='OP(O)(O)=S', type='cOP(=S)(OC)OC'), IFG(atomIds=(8,), atoms='n', type='cnc'), IFG(atomIds=(10,), atoms='Cl', type='cCl'), IFG(atomI

534: 5 fgs [IFG(atomIds=(2,), atoms='[nH]', type='c[nH]c'), IFG(atomIds=(4,), atoms='O', type='c=O'), IFG(atomIds=(5,), atoms='n', type='cn(c)C'), IFG(atomIds=(11,), atoms='O', type='c=O'), IFG(atomIds=(13,), atoms='Cl', type='cCl')]
535: 6 fgs [IFG(atomIds=(1,), atoms='O', type='cOC'), IFG(atomIds=(4,), atoms='Cl', type='cCl'), IFG(atomIds=(6,), atoms='Cl', type='cCl'), IFG(atomIds=(8,), atoms='Cl', type='cCl'), IFG(atomIds=(10,), atoms='Cl', type='cCl'), IFG(atomIds=(12,), atoms='O', type='cOC')]
536: 6 fgs [IFG(atomIds=(1,), atoms='O', type='cOC'), IFG(atomIds=(4,), atoms='Cl', type='cCl'), IFG(atomIds=(6,), atoms='Cl', type='cCl'), IFG(atomIds=(8,), atoms='Cl', type='cCl'), IFG(atomIds=(10,), atoms='Cl', type='cCl'), IFG(atomIds=(12,), atoms='O', type='cOC')]
537: 1 fgs [IFG(atomIds=(1,), atoms='Cl', type='CCl')]
538: 3 fgs [IFG(atomIds=(3,), atoms='N', type='cN(C)C'), IFG(atomIds=(9, 10, 11), atoms='O=[N+][O-]', type='c[N+](=O)[O-]'), IFG(atomIds=(19, 20, 21), atoms='O=[N+][O-]', 

821: 3 fgs [IFG(atomIds=(0, 1), atoms='C#C', type='C#CC'), IFG(atomIds=(4, 5, 6, 7), atoms='NC(=O)O', type='cNC(=O)OC'), IFG(atomIds=(13,), atoms='Cl', type='cCl')]
822: 1 fgs [IFG(atomIds=(3,), atoms='Cl', type='CCl')]
823: 1 fgs [IFG(atomIds=(0, 1, 2, 3), atoms='NC(N)=S', type='NC(N)=S')]
824: 1 fgs [IFG(atomIds=(0, 1), atoms='C#C', type='C#CC')]
825: 3 fgs [IFG(atomIds=(0,), atoms='O', type='cO'), IFG(atomIds=(8,), atoms='Cl', type='cCl'), IFG(atomIds=(6,), atoms='Cl', type='cCl')]
826: 4 fgs [IFG(atomIds=(3,), atoms='N', type='cN(C)C'), IFG(atomIds=(9, 10, 11), atoms='O=[N+][O-]', type='c[N+](=O)[O-]'), IFG(atomIds=(16, 17, 14, 15), atoms='NS(=O)=O', type='cS(N)(=O)=O'), IFG(atomIds=(20, 21, 22), atoms='O=[N+][O-]', type='c[N+](=O)[O-]')]
827: 4 fgs [IFG(atomIds=(0,), atoms='O', type='CO'), IFG(atomIds=(3,), atoms='O', type='CO'), IFG(atomIds=(5,), atoms='O', type='CO'), IFG(atomIds=(7,), atoms='O', type='CO')]
828: 6 fgs [IFG(atomIds=(0,), atoms='O', type='CO'), IFG(atomIds=(6,), 

In [5]:
# Richard hall 2017
# IFG main code
# Guillaume Godin 2017
# refine output function
# astex_ifg: identify functional groups a la Ertl, J. Cheminform (2017) 9:36

In [10]:
'Cn1cc(-c2ccccc2)c(=O)c(-c2cccc(C(F)(F)F)c2)c1'
m=Chem.MolFromSmiles('Cn1cc(-c2ccccc2)c(=O)c(-c2cccc(C(F)(F)F)c2)c1')
x=identify_functional_groups(m)


In [11]:
print(x)

[IFG(atomIds=(1,), atoms='n', type='cn(c)C'), IFG(atomIds=(11,), atoms='O', type='c=O'), IFG(atomIds=(19,), atoms='F', type='CF'), IFG(atomIds=(20,), atoms='F', type='CF'), IFG(atomIds=(21,), atoms='F', type='CF')]
